In [116]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from math import sqrt
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score
from sklearn import preprocessing

In [77]:
pd.set_option('display.max_rows', 1000)

In [78]:
df = pd.read_csv('processed_dataset.csv')

In [79]:
df['measure_date'] = pd.to_datetime(pd.to_datetime(df['measure_date']).dt.date)
df['departure_date'] = pd.to_datetime(pd.to_datetime(df['departure_date']).dt.date)
df['arrival_date'] = pd.to_datetime(pd.to_datetime(df['arrival_date']).dt.date)

In [99]:
df.price = df.price.astype(int)
df.previous_price_for_trip = df.previous_price_for_trip.astype(int)

df.measure_date = df.measure_date.astype(str)
df.departure_date = df.departure_date.astype(str)
df.arrival_date = df.arrival_date.astype(str)

In [100]:
df.head()

,departure_city,arrival_city,flight_duration,carrier_name,agent_name,flight_number,price,measure_date,departure_date,departure_hour,departure_minute,arrival_date,arrival_hour,arrival_minute,days_left_to_departure,previous_price_for_trip
0,moscow,budapest,165,aeroflot,aeroflot,2468,3710,2020-03-01,2020-05-01,20,10,2020-05-01,21,55,61,0
1,moscow,budapest,155,wizz air,wizz air,2490,4763,2020-03-01,2020-05-01,10,5,2020-05-01,11,40,61,0
2,moscow,budapest,130,lot,lot,5676,5306,2020-03-01,2020-05-01,18,5,2020-05-01,19,15,61,0
3,moscow,budapest,220,brussels airlines,brussels airlines,2846,1879,2020-03-01,2020-05-04,15,45,2020-05-04,18,25,64,0
4,moscow,budapest,155,wizz air,wizz air,2490,2177,2020-03-01,2020-05-04,10,5,2020-05-04,11,40,64,0


In [101]:
df.columns

Index(['departure_city', 'arrival_city', 'flight_duration', 'carrier_name',
       'agent_name', 'flight_number', 'price', 'measure_date',
       'departure_date', 'departure_hour', 'departure_minute', 'arrival_date',
       'arrival_hour', 'arrival_minute', 'days_left_to_departure',
       'previous_price_for_trip'],
      dtype='object')

In [102]:
features = list(filter(lambda x: 'price' != x, df.columns))

In [103]:
tscv = TimeSeriesSplit(n_splits=3)

In [104]:
features

['departure_city',
 'arrival_city',
 'flight_duration',
 'carrier_name',
 'agent_name',
 'flight_number',
 'measure_date',
 'departure_date',
 'departure_hour',
 'departure_minute',
 'arrival_date',
 'arrival_hour',
 'arrival_minute',
 'days_left_to_departure',
 'previous_price_for_trip']

In [118]:
df.dtypes

departure_city             int64
arrival_city               int64
flight_duration            int64
carrier_name               int64
agent_name                 int64
flight_number              int64
price                      int64
measure_date               int64
departure_date             int64
departure_hour             int64
departure_minute           int64
arrival_date               int64
arrival_hour               int64
arrival_minute             int64
days_left_to_departure     int64
previous_price_for_trip    int64
dtype: object

In [119]:
categorical_features = ['departure_city','arrival_city', 'carrier_name', 'agent_name', 'flight_number', 'measure_date', 'departure_date', 'arrival_date']

In [120]:
def rmse(real, predicted):
    return sqrt(mean_squared_error(real, predicted))

In [121]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols,query_cols,sorter=sidx)]

In [122]:
categorical_features_pos = column_index(df,categorical_features)

In [123]:
categorical_features_pos

array([ 0,  1,  3,  4,  5,  7,  8, 11])

In [124]:
lbl = preprocessing.LabelEncoder()
for col in categorical_features:
    df[col] = lbl.fit_transform(df[col].astype(str))

In [125]:
regressor = CatBoostRegressor(logging_level='Silent', random_state=0)

In [126]:
cv_rmse = []
for idx, split_idx_tuple in enumerate(tscv.split(df)):
    x_train_idx, x_test_idx = split_idx_tuple[0], split_idx_tuple[1]
    train, test = df.iloc[x_train_idx], df.iloc[x_test_idx]
    regressor.fit(train[features], train['price'], cat_features=categorical_features_pos, use_best_model=True)
    prediction = regressor.predict(test[features])
    current_rmse = rmse(prediction, test['price'])
    cv_rmse.append(current_rmse)
    print("MSE on {}th fold: {}".format(idx, current_rmse))

mean_rmse = sum(cv_rmse) / len(cv_rmse)
print("mean rmse: {}".format(mean_rmse))

You should provide test set for use best model. use_best_model parameter has been switched to false value.


Learning rate set to 0.079607
0:	learn: 2489.0200789	total: 99.1ms	remaining: 1m 38s
1:	learn: 2413.2703553	total: 128ms	remaining: 1m 3s
2:	learn: 2344.4910039	total: 148ms	remaining: 49.3s
3:	learn: 2282.3873830	total: 190ms	remaining: 47.2s
4:	learn: 2228.4925630	total: 219ms	remaining: 43.6s
5:	learn: 2173.4139988	total: 260ms	remaining: 43s
6:	learn: 2128.3089699	total: 300ms	remaining: 42.6s
7:	learn: 2083.2507697	total: 335ms	remaining: 41.5s
8:	learn: 2042.0479181	total: 371ms	remaining: 40.8s
9:	learn: 2005.8726273	total: 410ms	remaining: 40.5s
10:	learn: 1968.1876365	total: 446ms	remaining: 40.1s
11:	learn: 1937.3967518	total: 469ms	remaining: 38.7s
12:	learn: 1908.6730070	total: 488ms	remaining: 37.1s
13:	learn: 1885.3407825	total: 522ms	remaining: 36.8s
14:	learn: 1860.6865400	total: 556ms	remaining: 36.5s
15:	learn: 1841.8079962	total: 596ms	remaining: 36.7s
16:	learn: 1822.5779127	total: 635ms	remaining: 36.7s
17:	learn: 1806.1271788	total: 674ms	remaining: 36.8s
18:	lear

156:	learn: 1297.7294210	total: 6.12s	remaining: 32.9s
157:	learn: 1296.5970245	total: 6.17s	remaining: 32.9s
158:	learn: 1293.8739049	total: 6.21s	remaining: 32.9s
159:	learn: 1292.6431687	total: 6.26s	remaining: 32.9s
160:	learn: 1290.8136764	total: 6.29s	remaining: 32.8s
161:	learn: 1289.8153409	total: 6.34s	remaining: 32.8s
162:	learn: 1288.8113866	total: 6.4s	remaining: 32.8s
163:	learn: 1287.4710158	total: 6.43s	remaining: 32.8s
164:	learn: 1285.6848918	total: 6.44s	remaining: 32.6s
165:	learn: 1284.4248452	total: 6.47s	remaining: 32.5s
166:	learn: 1283.0997274	total: 6.53s	remaining: 32.6s
167:	learn: 1282.1311350	total: 6.58s	remaining: 32.6s
168:	learn: 1279.7335258	total: 6.62s	remaining: 32.5s
169:	learn: 1278.5268638	total: 6.67s	remaining: 32.5s
170:	learn: 1277.1045428	total: 6.71s	remaining: 32.5s
171:	learn: 1276.5942752	total: 6.78s	remaining: 32.7s
172:	learn: 1275.3263625	total: 6.82s	remaining: 32.6s
173:	learn: 1273.9412391	total: 6.84s	remaining: 32.5s
174:	learn:

307:	learn: 1173.5653370	total: 12.4s	remaining: 27.9s
308:	learn: 1173.1566029	total: 12.5s	remaining: 27.8s
309:	learn: 1172.6413950	total: 12.5s	remaining: 27.8s
310:	learn: 1172.0813652	total: 12.5s	remaining: 27.8s
311:	learn: 1170.4375473	total: 12.6s	remaining: 27.7s
312:	learn: 1169.7332696	total: 12.6s	remaining: 27.7s
313:	learn: 1169.0563671	total: 12.7s	remaining: 27.7s
314:	learn: 1168.2725232	total: 12.7s	remaining: 27.6s
315:	learn: 1167.4692368	total: 12.7s	remaining: 27.6s
316:	learn: 1167.1772384	total: 12.8s	remaining: 27.5s
317:	learn: 1166.3435163	total: 12.8s	remaining: 27.5s
318:	learn: 1166.0874073	total: 12.9s	remaining: 27.5s
319:	learn: 1165.3547254	total: 12.9s	remaining: 27.4s
320:	learn: 1164.6182002	total: 12.9s	remaining: 27.4s
321:	learn: 1164.3298210	total: 13s	remaining: 27.3s
322:	learn: 1164.0873789	total: 13s	remaining: 27.3s
323:	learn: 1163.8193613	total: 13.1s	remaining: 27.3s
324:	learn: 1163.1648773	total: 13.1s	remaining: 27.2s
325:	learn: 11

458:	learn: 1106.4291640	total: 18.4s	remaining: 21.7s
459:	learn: 1106.1407638	total: 18.4s	remaining: 21.6s
460:	learn: 1106.0176227	total: 18.5s	remaining: 21.6s
461:	learn: 1105.5086106	total: 18.5s	remaining: 21.6s
462:	learn: 1105.4244930	total: 18.6s	remaining: 21.5s
463:	learn: 1104.6542274	total: 18.6s	remaining: 21.5s
464:	learn: 1104.0017026	total: 18.6s	remaining: 21.5s
465:	learn: 1103.5712071	total: 18.7s	remaining: 21.4s
466:	learn: 1103.1643842	total: 18.7s	remaining: 21.4s
467:	learn: 1103.0483708	total: 18.8s	remaining: 21.3s
468:	learn: 1103.0065755	total: 18.8s	remaining: 21.3s
469:	learn: 1102.8207194	total: 18.9s	remaining: 21.3s
470:	learn: 1102.5699049	total: 18.9s	remaining: 21.3s
471:	learn: 1101.7498929	total: 19s	remaining: 21.2s
472:	learn: 1101.4788151	total: 19s	remaining: 21.2s
473:	learn: 1100.9685182	total: 19s	remaining: 21.1s
474:	learn: 1100.5944225	total: 19.1s	remaining: 21.1s
475:	learn: 1100.2873240	total: 19.1s	remaining: 21.1s
476:	learn: 1100

...

298:	learn: 1113.5805529	total: 27.5s	remaining: 1m 4s
299:	learn: 1113.0244679	total: 27.6s	remaining: 1m 4s
300:	learn: 1112.4062524	total: 27.6s	remaining: 1m 4s
301:	learn: 1112.1181296	total: 27.7s	remaining: 1m 4s
302:	learn: 1111.6784726	total: 27.8s	remaining: 1m 4s
303:	learn: 1110.0309515	total: 27.9s	remaining: 1m 3s
304:	learn: 1109.6388554	total: 28s	remaining: 1m 3s
305:	learn: 1109.2726404	total: 28.1s	remaining: 1m 3s
306:	learn: 1108.9706576	total: 28.3s	remaining: 1m 3s
307:	learn: 1108.4142644	total: 28.4s	remaining: 1m 3s
308:	learn: 1108.2393382	total: 28.5s	remaining: 1m 3s
309:	learn: 1107.7677457	total: 28.6s	remaining: 1m 3s
310:	learn: 1107.2432180	total: 28.7s	remaining: 1m 3s
311:	learn: 1107.0340621	total: 28.7s	remaining: 1m 3s
312:	learn: 1106.5525392	total: 28.9s	remaining: 1m 3s
313:	learn: 1105.9596955	total: 29s	remaining: 1m 3s
314:	learn: 1105.5591381	total: 29.1s	remaining: 1m 3s
315:	learn: 1105.0424064	total: 29.2s	remaining: 1m 3s
316:	learn: 11

450:	learn: 1062.5370789	total: 41.9s	remaining: 51s
451:	learn: 1061.6145958	total: 42s	remaining: 50.9s
452:	learn: 1061.3908178	total: 42.1s	remaining: 50.9s
453:	learn: 1061.1568829	total: 42.2s	remaining: 50.8s
454:	learn: 1060.7943031	total: 42.3s	remaining: 50.7s
455:	learn: 1060.6116814	total: 42.4s	remaining: 50.6s
456:	learn: 1060.2034680	total: 42.5s	remaining: 50.5s
457:	learn: 1059.7630041	total: 42.6s	remaining: 50.4s
458:	learn: 1059.6878074	total: 42.7s	remaining: 50.3s
459:	learn: 1059.4250524	total: 42.8s	remaining: 50.2s
460:	learn: 1059.1599987	total: 42.9s	remaining: 50.1s
461:	learn: 1058.8131366	total: 42.9s	remaining: 50s
462:	learn: 1058.5327356	total: 43s	remaining: 49.9s
463:	learn: 1058.3517417	total: 43.1s	remaining: 49.8s
464:	learn: 1058.2454622	total: 43.2s	remaining: 49.7s
465:	learn: 1057.6879175	total: 43.3s	remaining: 49.6s
466:	learn: 1057.5252212	total: 43.4s	remaining: 49.5s
467:	learn: 1057.3773817	total: 43.5s	remaining: 49.4s
468:	learn: 1057.1

602:	learn: 1021.1909765	total: 56.4s	remaining: 37.1s
603:	learn: 1021.0330635	total: 56.5s	remaining: 37s
604:	learn: 1020.9446911	total: 56.6s	remaining: 36.9s
605:	learn: 1020.8806104	total: 56.7s	remaining: 36.8s
606:	learn: 1020.3589090	total: 56.8s	remaining: 36.8s
607:	learn: 1020.2724606	total: 56.9s	remaining: 36.7s
608:	learn: 1020.1714187	total: 56.9s	remaining: 36.6s
609:	learn: 1019.9766381	total: 57.1s	remaining: 36.5s
610:	learn: 1019.6490779	total: 57.2s	remaining: 36.4s
611:	learn: 1019.5010432	total: 57.2s	remaining: 36.3s
612:	learn: 1019.0617925	total: 57.4s	remaining: 36.2s
613:	learn: 1018.5224251	total: 57.4s	remaining: 36.1s
614:	learn: 1018.4162877	total: 57.6s	remaining: 36s
615:	learn: 1018.1970907	total: 57.7s	remaining: 36s
616:	learn: 1017.9308895	total: 57.8s	remaining: 35.9s
617:	learn: 1017.7083153	total: 57.9s	remaining: 35.8s
618:	learn: 1017.4958136	total: 58s	remaining: 35.7s
619:	learn: 1017.3508278	total: 58s	remaining: 35.6s
620:	learn: 1017.146

754:	learn: 993.0372021	total: 1m 11s	remaining: 23.1s
755:	learn: 992.9707993	total: 1m 11s	remaining: 23s
756:	learn: 992.8071693	total: 1m 11s	remaining: 22.9s
757:	learn: 992.7863656	total: 1m 11s	remaining: 22.8s
758:	learn: 992.6109623	total: 1m 11s	remaining: 22.8s
759:	learn: 992.3588424	total: 1m 11s	remaining: 22.7s
760:	learn: 992.1933574	total: 1m 11s	remaining: 22.6s
761:	learn: 992.1189518	total: 1m 11s	remaining: 22.5s
762:	learn: 991.8888367	total: 1m 12s	remaining: 22.4s
763:	learn: 991.6778992	total: 1m 12s	remaining: 22.3s
764:	learn: 991.6316492	total: 1m 12s	remaining: 22.2s
765:	learn: 991.4729426	total: 1m 12s	remaining: 22.1s
766:	learn: 991.3545908	total: 1m 12s	remaining: 22s
767:	learn: 991.0953503	total: 1m 12s	remaining: 21.9s
768:	learn: 991.0547125	total: 1m 12s	remaining: 21.8s
769:	learn: 990.9982301	total: 1m 12s	remaining: 21.7s
770:	learn: 990.7311183	total: 1m 12s	remaining: 21.6s
771:	learn: 990.5845717	total: 1m 12s	remaining: 21.5s
772:	learn: 99

904:	learn: 969.2097291	total: 1m 25s	remaining: 8.98s
905:	learn: 969.1084885	total: 1m 25s	remaining: 8.89s
906:	learn: 968.9198608	total: 1m 25s	remaining: 8.79s
907:	learn: 968.8111220	total: 1m 25s	remaining: 8.7s
908:	learn: 968.6991517	total: 1m 26s	remaining: 8.61s
909:	learn: 968.6895819	total: 1m 26s	remaining: 8.52s
910:	learn: 968.6001483	total: 1m 26s	remaining: 8.43s
911:	learn: 968.4279106	total: 1m 26s	remaining: 8.33s
912:	learn: 968.3761873	total: 1m 26s	remaining: 8.24s
913:	learn: 968.1635317	total: 1m 26s	remaining: 8.14s
914:	learn: 968.1102960	total: 1m 26s	remaining: 8.05s
915:	learn: 968.1027700	total: 1m 26s	remaining: 7.95s
916:	learn: 968.0857612	total: 1m 26s	remaining: 7.85s
917:	learn: 967.9618827	total: 1m 26s	remaining: 7.76s
918:	learn: 967.9574835	total: 1m 27s	remaining: 7.67s
919:	learn: 967.8152843	total: 1m 27s	remaining: 7.57s
920:	learn: 967.5337761	total: 1m 27s	remaining: 7.48s
921:	learn: 967.4107521	total: 1m 27s	remaining: 7.39s
922:	learn: